This is a baseline model to detect Melanoma in the images given in the dataset. Here I have used subset of training data (3500 images). I used 3 pre-trained models which are VGG16, Resnet50 and Efficientnet B7, and ensembled the results using weighted average.

The process will be consisting of below steps.

1. Importing the required libraries
2. Loading and preparng the images
3. Preprocessing
4. Defining and training the model8
5. Getting predictions for the model
6. Repeating step 4,5 for other models
7. Blending the scores from all 3 models
8. Submission

#  Importing Libraries

In [ ]:
import numpy as np
import pandas as pd

import keras
from keras.preprocessing import image
from keras.utils import to_categorical

from keras.applications.vgg16 import VGG16,preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,BatchNormalization,Activation

from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import Adam, SGD, RMSprop
from tensorflow.python.keras import backend as K

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2

# Exploring the data

In [ ]:
# Directory Listings for train and test images

train_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'

In [ ]:
# Reading the csv files

train = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')
submission=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')

In [ ]:
# Exploring the content of train file
train.head()

In [ ]:
# Comparing the number of records in both categories
train['target'].value_counts()

The data is highly imbalanced as only 584 records belong to class 1 (Malignant).
Hence we will be taking a sample of 3000 images from benign cases, along with all 584 malignant cases

In [ ]:
train_benign = train[train['target']==0].sample(3000)
train_malign = train[train['target']==1]
train_samples = pd.concat([train_benign,train_malign])
train_samples.reset_index()

# Preparing the train and test data

In [ ]:
# Training data
train_labels = []
train_images =[]

for i in range(train_samples.shape[0]):
    train_images.append(train_dir+train_samples['image_name'].iloc[i]+'.jpg')
    train_labels.append(train_samples['target'].iloc[i])

df_train = pd.DataFrame(train_images)
df_train.columns =['images']
df_train['target'] = train_labels

In [ ]:
# Test data
test_images =[]
for i in range(test.shape[0]):
    test_images.append(test_dir+test['image_name'].iloc[i]+'.jpg')

df_test = pd.DataFrame(test_images)
df_test.columns = ['images']

In [ ]:
# Splitting the train data further into train and validation sets
X_train, X_val, y_train,y_val = train_test_split(df_train['images'],df_train['target'],test_size=0.2,random_state=0)

train = pd.DataFrame(X_train)
train.columns = ['images']
train['target']=y_train

validation = pd.DataFrame(X_val)
validation.columns = ['images']
validation['target']=y_val

# Helper Functions

In [ ]:
def get_predictions(model,sub_df):
    target=[]
    for path in df_test['images']:
        img=cv2.imread(str(path))
        img = cv2.resize(img, (224,224))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)/255.
        img=np.reshape(img,(1,224,224,3))
        prediction=model.predict(img)
        target.append(prediction[0][0])
    
    sub_df['target']=target
    return sub_df

# Data Preprocessing

Data Processing will include following tasks

1. Normalization of pixel values to range of 0 to 1
2. Reshaping of the images
3. Data augumentation for training images

In [ ]:
train_datagen = ImageDataGenerator(preprocess_input,rescale=1./255,rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,horizontal_flip=True)

val_datagen = ImageDataGenerator(preprocess_input,rescale=1./255)

image_size = 224

train_generator = train_datagen.flow_from_dataframe(
                    train,
                    x_col='images',
                    y_col ='target',
                    target_size=(image_size,image_size),
                    batch_size=8,
                    shuffle=True,
                    class_mode='raw')

validation_generator = val_datagen.flow_from_dataframe(
                    validation,
                    x_col='images',
                    y_col ='target',
                    target_size=(image_size,image_size),
                    batch_size=8,
                    shuffle=False,
                    class_mode='raw')

# Modeling

**1. VGG16 Pre-trained Model**

In [ ]:
def vgg16_model(num_classes=None):
    model = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
    x = Flatten()(model.output)
    output = Dense(1,activation='sigmoid')(x)
    model = Model(model.input,output)
    
    return model

In [ ]:
vgg_conv = vgg16_model(1)

Using focal loss instead of binary_cropssentory because of class imbalance

In [ ]:
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [ ]:
# Defining the optimizer and compiling the model
opt = Adam(lr=1e-5)
vgg_conv.compile(loss=focal_loss(),optimizer=opt,metrics=[keras.metrics.AUC()])

In [ ]:
# Denining the num of epochs, batch_size and steps for training and validation
nb_epochs = 2
batch_size=8
nb_train_steps = train.shape[0]//batch_size  # // rounds off the result of division
nb_validation_steps = validation.shape[0]//batch_size
print("Number of training and validation steps are {} and {}".format(nb_train_steps,nb_validation_steps))

In [ ]:
# Fitting the model
vgg_conv.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_steps)

In [ ]:
# Getting the predictions for test data

sub_dfs = []

sub_vgg16 = submission.copy()
sub_vgg16 = get_predictions(vgg_conv,sub_vgg16)
sub_vgg16.head()

In [ ]:
sub_vgg16.to_csv('submission_vgg16.csv',index=False)

In [ ]:
df_vgg16 = pd.read_csv('../input/subvgg16csv/submission_vgg16.csv')
df_vgg16.head()

In [ ]:
sub_dfs = []
sub_dfs.append(df_vgg16)

**2. Resnet50 Pretrained Model**

In [ ]:
def resnet50_model(num_classes=None):
    model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3),pooling='avg')
    x = Flatten()(model.output)
    output = Dense(1,activation='sigmoid')(x)
    model = Model(model.input,output)
    
    return model

In [ ]:
resnet_conv = resnet50_model(1)

In [ ]:
# Denining the optimizer and compiling the model
opt = Adam(lr=1e-5)
resnet_conv.compile(loss=focal_loss(),optimizer=opt,metrics=[keras.metrics.AUC()])

In [ ]:
# Denining the num of epochs, batch_size and steps for training and validation
nb_epochs = 2
batch_size=8
nb_train_steps = train.shape[0]//batch_size  # // rounds off the result of division
nb_validation_steps = validation.shape[0]//batch_size
print("Number of training and validation steps are {} and {}".format(nb_train_steps,nb_validation_steps))

In [ ]:
# Fitting the model
resnet_conv.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_steps)

In [ ]:
# Getting the predictions for test data


sub_resnet = submission.copy()
sub_resnet = get_predictions(resnet_conv,sub_resnet)
sub_resnet.head()

In [ ]:
sub_resnet.to_csv('submission_resnet.csv',index=False)

In [ ]:
df_resnet = pd.read_csv('../input/subresnetcsv/submission_resnet.csv')
sub_dfs.append(df_resnet)

**2. EfficientNet B7 Pretrained Model**

In [ ]:
!pip install -q efficientnet
import efficientnet.tfkeras as efn

In [ ]:
def efficientnet_model(num_classes=None):
    
    model = keras.Sequential([
        
            efn.EfficientNetB7(
            weights='imagenet',
            include_top=False,
            input_shape=(224,224,3)),
        keras.layers.GlobalAveragePooling2D(), #Works like flatten layer but adds pooling to it to optimize data before feeding to FC layer
        keras.layers.Dense(1024,activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(512,activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(256,activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(128,activation='relu'),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(1,activation='sigmoid')
        
    ])
    return model

In [ ]:
efficientnet_conv = efficientnet_model(1)

In [ ]:
# Defining the optimizer and compiling the model
opt = Adam(lr=1e-5)
efficientnet_conv.compile(loss=focal_loss(),optimizer=opt,metrics=[keras.metrics.AUC()])

In [ ]:
# Defining the num of epochs, batch_size and steps for training and validation
nb_epochs = 2
batch_size=8
nb_train_steps = train.shape[0]//batch_size  # // rounds off the result of division
nb_validation_steps = validation.shape[0]//batch_size
print("Number of training and validation steps are {} and {}".format(nb_train_steps,nb_validation_steps))

In [ ]:
# Fitting the model
efficientnet_conv.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_steps)

In [ ]:
# Getting the predictions for test data


sub_efficientnet = submission.copy()
sub_efficientnet = get_predictions(efficientnet_conv,sub_efficientnet)
sub_efficientnet.head()

In [ ]:
sub_efficientnet.to_csv('submission_efficientnet.csv',index=False)

In [ ]:
df_efficient = pd.read_csv('../input/subefficientnetcsv/submission_efficientnet.csv')
sub_dfs.append(df_efficient)

# Blending Scores from All Models

In [ ]:
from scipy.stats import rankdata

In [ ]:
#Ranking results from all models
for i in range(3) :
    sub_dfs[i]['target'] = rankdata(sub_dfs[i]['target'], method='min')

Giving more weightage to VGG16 since it gave better score on validation data

In [ ]:
sub_dfs[0]['target'] = sub_dfs[0]['target']*0.8 + sub_dfs[1]['target']*0.1 + sub_dfs[2]['target']*0.1
sub_dfs[0].head()

# Submitting Final Predictions

In [ ]:
sub_dfs[0].to_csv('submission_blending.csv' , index = False)